<a href="https://colab.research.google.com/github/akkinepally/email_classification/blob/main/email_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# import libraries 
import pandas as pd
import re  
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk
import spacy
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [9]:
# load data 
email_train_dataframe = pd.read_csv('train.csv', encoding='utf-8')
email_test_dataframe = pd.read_csv('test.csv', encoding='utf-8')


In [10]:
email_train_dataframe 

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,2,0,Finally a transparant silicon case ^^ Thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...
3,4,0,I'm wired I know I'm George I was made that wa...
4,5,1,What amazing service! Apple won't even talk to...
...,...,...,...
7915,7916,0,Live out loud #lol #liveoutloud #selfie #smile...
7916,7917,0,We would like to wish you an amazing day! Make...
7917,7918,0,Helping my lovely 90 year old neighbor with he...
7918,7919,0,Finally got my #smart #pocket #wifi stay conne...


In [32]:
nlp = spacy.load('en', disable=['parser', 'ner'])
corpus = []
lem = WordNetLemmatizer()
punctuation = ["'#$%^&*()!@~"]
for count in range(len(email_train_dataframe['tweet'])):
  clean_tweet = re.sub('http\S+', '', email_train_dataframe['tweet'][count] )
  clean_tweet = clean_tweet.lower()
  clean_tweet = re.sub('[^a-zA-Z]', ' ', clean_tweet )
  clean_tweet = re.sub("'#$%^&*()!@~", ' ', clean_tweet)
  clean_tweet = clean_tweet.split()
  clean_tweet = [lem.lemmatize(word) for word in clean_tweet if word not in set(stopwords.words('english'))]
  clean_tweet = " ".join(clean_tweet)
  corpus.append(clean_tweet)




In [19]:
email_train_dataframe['clean_tweet'] = corpus

In [23]:
email_train_dataframe

,id,label,tweet,clean_tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...,fingerprint pregnancy test android apps beauti...
1,2,0,Finally a transparant silicon case ^^ Thanks t...,finally transparant silicon case thanks uncle ...
2,3,0,We love this! Would you go? #talk #makememorie...,love would go talk makememories unplug relax i...
3,4,0,I'm wired I know I'm George I was made that wa...,wired know george made way iphone cute daventr...
4,5,1,What amazing service! Apple won't even talk to...,amazing service apple even talk question unles...
...,...,...,...,...
7915,7916,0,Live out loud #lol #liveoutloud #selfie #smile...,live loud lol liveoutloud selfie smile sony mu...
7916,7917,0,We would like to wish you an amazing day! Make...,would like wish amazing day make every minute ...
7917,7918,0,Helping my lovely 90 year old neighbor with he...,helping lovely year old neighbor ipad morning ...
7918,7919,0,Finally got my #smart #pocket #wifi stay conne...,finally got smart pocket wifi stay connected a...


In [10]:
! pip install tensorflow==1.15
! pip install tensorflow-hub>=0.6.0

     |████████████████████████████████| 412.3MB 25kB/s 
     |████████████████████████████████| 51kB 6.1MB/s 
     |████████████████████████████████| 3.8MB 28.8MB/s 
     |████████████████████████████████| 512kB 41.0MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7542 sha256=33fdbe871a6b469790f980939830ce715342e76b99b8a35a8b0acd857ad99862
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tensorflow-estimator-2.3.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully 

In [3]:
import tensorflow_hub as hub
import tensorflow as tf

elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)


In [26]:
def elmo_vector(text):
  embed = elmo(text, signature="default", as_dict=True)["elmo"]
  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    return sess.run(tf.reduce_mean(embed, 1))

In [21]:
train_batch = [email_train_dataframe[i:i+100] for i in range(0,email_train_dataframe.shape[0], 100)]

[     id  ...                                        clean_tweet
 0     1  ...  fingerprint pregnancy test android apps beauti...
 1     2  ...  finally transparant silicon case thanks uncle ...
 2     3  ...  love would go talk makememories unplug relax i...
 3     4  ...  wired know george made way iphone cute daventr...
 4     5  ...  amazing service apple even talk question unles...
 ..  ...  ...                                                ...
 95   96  ...  new model byhilalak earphone urbanears handmad...
 96   97  ...  photo itunes vicio nellyfurtado nelstar playin...
 97   98  ...  see many feature know iphone iphone yourtechco...
 98   99  ...  happiness box yey phone sony xperia sonyxperia...
 99  100  ...  feliz bado con caf saturday coffee pantry iphonex
 
 [100 rows x 4 columns],
       id  ...                                        clean_tweet
 100  101  ...  dreamy effect created straight iphone impressi...
 101  102  ...  finalllllly note note samsung galaxy galaxyno

In [27]:
elmo_train = [elmo_vector(x['clean_tweet']) for x in train_batch]

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [28]:
import numpy as np 
ELMO_TRAIN_COMPLETE = np.concatenate(elmo_train, axis=0)

In [29]:

from sklearn.model_selection import train_test_split

xtrain, xvalid, ytrain, yvalid = train_test_split(ELMO_TRAIN_COMPLETE, 
                                                 email_train_dataframe['label'],  
                                                  random_state=42, 
                                                  test_size=0.2)

In [30]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

lreg = LogisticRegression()
lreg.fit(xtrain, ytrain)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [31]:
pred_val = lreg.predict(xvalid)
f1_score(yvalid,pred_val)

0.7530266343825665